In [1]:
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
from time import sleep
import json
import config

league_id = 15728

teams = pd.read_csv('teams.csv')

In [2]:
def get_top_elo(date=None):
    top_teams = pd.DataFrame()

    fir = ['team','elo_(k=32)','elo_(k=64)','glicko_1','glicko_2']
    sec = ['', 'avg_7d', 'δ7d', 'avg_30d', 'δ30d', 'rating', 'μ', 'σ', 'δrat.7d']
    
    url = f"https://www.datdota.com/ratings"
    page = requests.get(url)

    soup = bs(page.text,'html.parser')
    table_body = soup.find('table')

    row_data = []
    for row in table_body.find_all('tr'):
        col = row.find_all('td')
        col = [ele.text.strip() for ele in col]
        row_data.append(col)

    idx = pd.MultiIndex(levels=[fir,sec],codes=[[0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,4],[0,5,1,2,3,4,5,1,2,3,4,5,6,7,8,5,6,7,8]], names=['lvl1','lvl2'])

    top_teams = pd.DataFrame(row_data[2:],columns=idx)

    top_teams['team'] = top_teams['team'].apply(lambda x: x.split('\n')[0])
    if date:
        top_teams['date'] = date

    #Flatten multiindex
    top_teams.columns = ['_'.join(x) for x in top_teams.columns.to_flat_index()]
    
    #Fix team column
    top_teams.rename({'team_':'team'},axis=1,inplace=True)
    
    return top_teams

In [3]:
df = get_top_elo()

In [4]:
df.head(20)

,team,elo_(k=32)_rating,elo_(k=32)_avg_7d,elo_(k=32)_δ7d,elo_(k=32)_avg_30d,elo_(k=32)_δ30d,elo_(k=64)_rating,elo_(k=64)_avg_7d,elo_(k=64)_δ7d,elo_(k=64)_avg_30d,elo_(k=64)_δ30d,glicko_1_rating,glicko_1_μ,glicko_1_σ,glicko_1_δrat.7d,glicko_2_rating,glicko_2_μ,glicko_2_σ,glicko_2_δrat.7d
0,Team Spirit,1178.07,1157.10,+23.72,1138.27,+62.39,1273.75,1230.30,+44.16,1193.62,+130.78,,,,-,1995.61,2103.09,42.99,-3.24
1,Gaimin Gladiators,1186.27,1227.31,-56.93,1266.46,-168.35,1185.92,1270.11,-113.73,1325.39,-300.90,,,,-,1987.39,2090.10,41.08,-3.38
2,BetBoom Team,1141.40,1138.72,+12.09,1133.12,+85.88,1186.38,1171.47,+38.80,1197.20,+112.30,,,,-,1955.77,2059.52,41.50,-3.36
3,Team Liquid,1103.24,1122.23,-53.10,1201.37,-117.76,1143.16,1135.19,-21.60,1254.53,-137.05,,,,-,1928.76,2040.15,44.56,-3.24
4,Quest Esports,1160.55,1164.48,-7.92,1128.78,+99.84,1277.93,1284.74,-13.71,1227.00,+166.41,,,,-,1924.75,2036.21,44.59,-3.11
5,Tundra Esports,1170.56,1222.94,-86.31,1240.89,-45.29,1178.17,1280.06,-181.97,1329.03,-96.99,,,,-,1920.51,2032.81,44.92,-3.16
6,Shopify Rebellion,1138.15,1165.88,-63.34,1174.48,-0.07,1175.29,1207.53,-89.39,1212.57,+45.40,,,,-,1894.81,2004.02,43.68,-3.18
7,Team Aster,1138.32,1111.66,+44.02,1145.97,+19.43,1214.48,1158.23,+100.78,1220.04,+22.48,,,,-,1884.78,2002.14,46.95,-3.04
8,9Pandas,1224.67,1178.20,+73.11,1169.76,+42.18,1351.25,1266.90,+131.39,1242.43,+73.00,,,,-,1880.80,1984.52,41.49,-3.38
9,OG,1111.58,1121.73,-27.04,1121.66,-46.65,1165.52,1186.51,-55.73,1168.72,-49.84,,,,-,1875.47,1985.10,43.86,-3.24


In [5]:
def get_teams(pages=1):
    df = pd.DataFrame()
    for page in tqdm(range(pages)):
        request = requests.request('GET',f"https://api.opendota.com/api/teams?page={page}")
        data = request.json()
        df = pd.concat([df,pd.DataFrame(data)],ignore_index=True)
        sleep(1)
    df['team_id'] = df['team_id'].astype(int)
    return df

In [6]:
def get_players(teams,active=True):
    dfs = []
    
    for team in tqdm(teams):
        request = requests.get(f"https://api.opendota.com/api/teams/{team}/players?api_key={config.api_key}")
        data = request.json()
        df = pd.DataFrame(data)
        df['team_id'] = team
        dfs.append(df)
    df = pd.concat(dfs,ignore_index=True)
    
    if active:
        df = df[df['is_current_team_member']==True]
        df = df.drop('is_current_team_member',axis=1).reset_index(drop=True)
    return df

In [7]:
players = get_players(teams['id'])

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:42<00:00,  2.12s/it]


In [11]:
account = players['account_id'][4]

In [32]:
def get_player_matches(player, matches=50):
    # 7.34 matches
    request = requests.get(f"https://api.opendota.com/api/players/{account}/matches?api_key={config.api_key}&game_mode=2&patch=53&limit={matches}")
    data = request.json()
    if len(data) < matches:
        # 7.33 matches
        diff = matches - len(data)
        request = requests.get(f"https://api.opendota.com/api/players/{account}/matches?api_key={config.api_key}&game_mode=2&patch=52&limit={diff}")
        sec_data = request.json()
        data = data + sec_data
        
    return pd.DataFrame(data)

In [33]:
dfs = []
for account in tqdm(players['account_id']):
    data = get_player_matches(account)
    dfs.append(data)
    
matches = pd.concat(dfs,ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [09:26<00:00,  4.97s/it]


In [36]:
players.to_csv('raw_players.csv',index=False)
matches.to_csv('raw_matches.csv',index=False)

In [37]:
unique_matches = matches['match_id'].unique()

In [38]:
len(unique_matches)

994

In [39]:
match = unique_matches[0]

In [40]:
request = requests.get(f"https://api.opendota.com/api/matches/{match}?api_key={config.api_key}")
data = request.json()

In [48]:
data['players'][0]['purchase_time']

{'magic_stick': -89,
 'branches': -178,
 'smoke_of_deceit': 4794,
 'tango': 51,
 'blood_grenade': -89,
 'flask': 31,
 'ward_sentry': 30573,
 'wind_lace': 222,
 'magic_wand': 224,
 'boots': 341,
 'ward_observer': 16135,
 'tpscroll': 5114,
 'infused_raindrop': 427,
 'ring_of_regen': 583,
 'tranquil_boots': 610,
 'void_stone': 870,
 'energy_booster': 870,
 'clarity': 932,
 'aether_lens': 1048,
 'ogre_axe': 1499,
 'great_famango': 1545,
 'gem': 1672,
 'staff_of_wizardry': 1907,
 'blade_of_alacrity': 4138,
 'point_booster': 2015,
 'ultimate_scepter': 2143,
 'shadow_amulet': 2262,
 'cloak': 2262,
 'glimmer_cape': 2416}

In [22]:
teamdf = get_teams(20)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:25<00:00,  1.28s/it]


In [23]:
test = pd.merge(df,teamdf,how='left',left_on='team',right_on='name')

In [31]:
df[df['team'].str.contains('LGD')]

,team,elo_(k=32)_rating,elo_(k=32)_avg_7d,elo_(k=32)_δ7d,elo_(k=32)_avg_30d,elo_(k=32)_δ30d,elo_(k=64)_rating,elo_(k=64)_avg_7d,elo_(k=64)_δ7d,elo_(k=64)_avg_30d,elo_(k=64)_δ30d,glicko_1_rating,glicko_1_μ,glicko_1_σ,glicko_1_δrat.7d,glicko_2_rating,glicko_2_μ,glicko_2_σ,glicko_2_δrat.7d


In [24]:
test[test['team_id'].isin(teams['id'])]

,team,elo_(k=32)_rating,elo_(k=32)_avg_7d,elo_(k=32)_δ7d,elo_(k=32)_avg_30d,elo_(k=32)_δ30d,elo_(k=64)_rating,elo_(k=64)_avg_7d,elo_(k=64)_δ7d,elo_(k=64)_avg_30d,...,glicko_2_σ,glicko_2_δrat.7d,team_id,rating,wins,losses,last_match_time,name,tag,logo_url
0,Team Spirit,1178.07,1157.10,+23.72,1138.27,+62.39,1273.75,1230.30,+44.16,1193.62,...,41.69,-3.35,7119388.0,1652.28,446.0,317.0,1.695575e+09,Team Spirit,TSpirit,https://steamusercontent-a.akamaihd.net/ugc/18...
1,Gaimin Gladiators,1186.27,1227.31,-56.93,1266.46,-168.35,1185.92,1270.11,-113.73,1325.39,...,39.73,-3.50,8599101.0,1517.25,289.0,174.0,1.695386e+09,Gaimin Gladiators,GG,https://steamusercontent-a.akamaihd.net/ugc/18...
3,BetBoom Team,1141.40,1138.72,+12.09,1133.12,+85.88,1186.38,1171.47,+38.80,1197.20,...,40.16,-3.47,8255888.0,1528.82,230.0,207.0,1.695558e+09,BetBoom Team,BetBoom,https://steamusercontent-a.akamaihd.net/ugc/20...
4,Team Liquid,1103.24,1122.23,-53.10,1201.37,-117.76,1143.16,1135.19,-21.60,1254.53,...,43.26,-3.34,2163.0,1429.91,1306.0,877.0,1.695208e+09,Team Liquid,Liquid,https://steamcdn-a.akamaihd.net/apps/dota2/ima...
5,Quest Esports,1160.55,1164.48,-7.92,1128.78,+99.84,1277.93,1284.74,-13.71,1227.00,...,43.34,-3.20,8894818.0,1418.86,137.0,79.0,1.695294e+09,Quest Esports,Quest,https://steamusercontent-a.akamaihd.net/ugc/20...
6,Tundra Esports,1170.56,1222.94,-86.31,1240.89,-45.29,1178.17,1280.06,-181.97,1329.03,...,43.65,-3.26,8291895.0,1426.81,296.0,192.0,1.695472e+09,Tundra Esports,Tundra,https://steamusercontent-a.akamaihd.net/ugc/20...
7,Shopify Rebellion,1138.15,1165.88,-63.34,1174.48,-0.07,1175.29,1207.53,-89.39,1212.57,...,42.41,-3.28,39.0,1460.63,1384.0,852.0,1.695575e+09,Shopify Rebellion,SR,https://steamcdn-a.akamaihd.net/apps/dota2/ima...
11,9Pandas,1224.67,1178.20,+73.11,1169.76,+42.18,1351.25,1266.90,+131.39,1242.43,...,40.14,-3.50,7422789.0,1390.87,437.0,375.0,1.695398e+09,9Pandas,9Pandas,https://steamusercontent-a.akamaihd.net/ugc/20...
18,Talon,1134.02,1108.84,+50.10,1117.37,+11.25,1204.88,1154.47,+106.31,1175.04,...,44.83,-3.07,8597976.0,1385.56,210.0,146.0,1.695208e+09,Talon,TLN,https://steamusercontent-a.akamaihd.net/ugc/20...
19,Entity,981.17,1009.53,-49.67,1030.85,-87.70,991.87,1042.42,-88.04,1082.12,...,45.69,-3.02,8605863.0,1434.74,204.0,167.0,1.695407e+09,Entity,Entity,https://steamusercontent-a.akamaihd.net/ugc/20...


In [14]:
data = requests.get(f"https://api.opendota.com/api/leagues/{league_id}/teams")

In [15]:
data.json()

[]

In [6]:
league_data = pd.DataFrame(data.json())

In [8]:
league_data[league_data['name'].str.contains('International')]

,leagueid,ticket,banner,tier,name
53,65005,econ/leagues/subscriptions_westqualifier_ingame,econ/leagues/subscriptions_westqualifier_ingame,professional,The International West Qualifiers
75,65004,econ/leagues/subscriptions_eastqualifier_ingame,econ/leagues/subscriptions_eastqualifier_ingame,professional,The International East Qualifiers
315,1964,None,None,excluded,Dota International Clan Wars - Season 1
503,600,None,None,professional,The International 2014
808,3623,None,None,excluded,Dota International Clan Wars - Season 5
858,2163,None,None,excluded,Dota International Clan Wars - Season 2
976,2733,None,None,professional,The International 2015
1181,3301,None,None,excluded,Dota International's Clan Wars - Season 4
1216,4664,None,None,premium,The International 2016
1414,4768,None,None,excluded,The International 2016 Open Qualifiers


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
from time import sleep
import json